# Онтологический анализатор нормативных документов
## Генерация и оценка онтологий с использованием LLM

Импорт основных компонентов из модуля `ontology_system`:

In [ ]:
# Установка зависимостей
!pip install owlready2 openai scikit-learn matplotlib

In [ ]:
import json
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import warnings

# Импорт основных классов из модуля
from src.llm_ontology import OntologyGenerator, OntologyEvaluator

# Настройка отображения
%matplotlib inline
warnings.filterwarnings("ignore", category=UserWarning)
plt.style.use('ggplot')

## 1. Подготовка тестовых данных

In [ ]:
law_text = """
Закон о персональных данных

Статья 1. Основные понятия
1. Персональные данные - любая информация, относящаяся к прямо или косвенно определенному физическому лицу.
2. Оператор - государственный орган, юридическое лицо, обрабатывающее персональные данные.

Статья 2. Принципы обработки
1. Обработка должна осуществляться на законной основе.
2. Объем данных должен соответствовать заявленным целям.

Статья 3. Права субъекта
1. Субъект имеет право на доступ к своим данным.
2. Субъект может требовать удаления данных.
"""

with open("test_law.txt", "w", encoding="utf-8") as f:
    f.write(law_text)

# Gold standard для оценки
gold_standard = {
    "classes": ["ПерсональныеДанные", "Оператор", "СубъектДанных"],
    "relations": [
        {"source": "Оператор", "target": "ПерсональныеДанные", "type": "обрабатывает"},
        {"source": "СубъектДанных", "target": "ПерсональныеДанные", "type": "имеет"}
    ]
}

with open("gold_standard.json", "w", encoding="utf-8") as f:
    json.dump(gold_standard, f, ensure_ascii=False, indent=2)

## 2. Генерация онтологии

In [ ]:
# Инициализация генератора (в production укажите реальный API ключ)
# openai.api_key = "ваш_api_ключ"
generator = OntologyGenerator(llm_model="gpt-3.5-turbo")

# Генерация онтологии
display(Markdown("### Генерация онтологии из текста закона..."))
ontology = generator.generate_from_text(law_text, "privacy_law")
ontology.save(file="privacy_law.owl", format="rdfxml")

# Генерация вопросов компетентности
display(Markdown("### Генерация вопросов компетентности (CQ)..."))
cq_list = generator.generate_competency_questions(ontology, num_questions=5)

display(Markdown("#### Сгенерированные CQ:"))
for i, cq in enumerate(cq_list, 1):
    display(Markdown(f"{i}. {cq}"))

## 3. Визуализация онтологии

In [ ]:
# Инициализация оценщика
evaluator = OntologyEvaluator("privacy_law.owl")

# Визуализация иерархии классов
display(Markdown("### Визуализация иерархии классов"))
evaluator.visualize_hierarchy()

## 4. Оценка качества онтологии

In [ ]:
# Вычисление метрик
display(Markdown("### Расчет метрик качества"))

# Покрытие CQ (предположим, что ответили на 4 из 5)
cq_coverage = evaluator.calculate_cq_coverage(cq_list, cq_list[:4])

# Индекс ошибок
error_index = evaluator.detect_ontology_errors()

# F1-Score
with open("gold_standard.json", "r", encoding="utf-8") as f:
    gold_standard = json.load(f)
f1 = evaluator.evaluate_f1(gold_standard, ontology)

# Глубина иерархии
depth = evaluator.calculate_hierarchy_depth()

# Семантическая оценка
g_eval_prompt = """Оцените от 1 до 5, насколько класс '{class_name}' соответствует документу:
Определение класса: {class_definition}
Контекст документа: {document_text}"""
semantic_score = evaluator.semantic_evaluation(law_text, g_eval_prompt)

# Вывод результатов
metrics_data = {
    "Метрика": ["Покрытие CQ", "Индекс ошибок", "F1-Score", "Глубина иерархии", "Семантическая оценка"],
    "Значение": [cq_coverage, error_index, f1, depth, semantic_score],
    "Целевое": [0.85, 0.05, 0.8, "4-6", 4.0]
}

display(Markdown("#### Таблица метрик:"))
display(pd.DataFrame(metrics_data).style\
    .bar(subset=['Значение'], align='mid', color=['#d65f5f', '#5fba7d']))

## 5. Визуализация результатов

In [ ]:
# Графическое представление метрик
plt.figure(figsize=(10, 5))

metrics = ['CQ Coverage', 'Error Index', 'F1-Score', 'Semantic Score']
values = [cq_coverage, 1-error_index, f1, semantic_score/5]  # Нормализация
targets = [0.85, 0.95, 0.8, 0.8]  # Целевые значения (нормализованные)

x = range(len(metrics))
width = 0.35

plt.bar(x, values, width, label='Текущие')
plt.bar([p + width for p in x], targets, width, label='Целевые')

plt.xticks([p + width/2 for p in x], metrics)
plt.title("Сравнение метрик с целевыми значениями")
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Генерация отчета

In [ ]:
# Формирование полного отчета
report = {
    "metadata": {
        "project": "Анализ закона о персональных данных",
        "date": pd.Timestamp.now().strftime("%Y-%m-%d")
    },
    "metrics": {
        "cq_coverage": cq_coverage,
        "error_index": error_index,
        "f1_score": f1,
        "hierarchy_depth": depth,
        "semantic_score": semantic_score
    },
    "recommendations": evaluator.generate_recommendations()
}

# Сохранение отчета
with open("ontology_report.json", "w", encoding="utf-8") as f:
    json.dump(report, f, ensure_ascii=False, indent=2)

display(Markdown("### Отчет успешно сохранен в `ontology_report.json`"))
display(Markdown("#### Ключевые рекомендации:"))
for rec in report["recommendations"]:
    display(Markdown(f"- {rec}"))